In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib

In [3]:
import pandas as pd
import numpy as np

# Read the CSV and Perform Basic Data Cleaning

In [4]:
red_wine_df = pd.read_csv("resources/winequality-red.csv", sep=';')
white_wine_df = pd.read_csv("resources/winequality-white.csv", sep=';')

In [5]:
#combine all white and red wine data
baseline_red_df = red_wine_df.copy()
baseline_white_df = white_wine_df.copy()
baseline_wine_df = pd.concat([baseline_red_df, baseline_white_df])
baseline_wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Select your features (columns)

In [6]:
# Set features. Drop koi disposition, y value. 
X = baseline_wine_df.drop(columns='quality')
X.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


# Create a Train Test Split

Use `quality` for the y values

In [7]:
from sklearn.model_selection import train_test_split
#80% train, 20% test. y is this one column
y = baseline_wine_df['quality']
#random state 42 will have same picks for x test and y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [8]:
X_train.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
139,7.8,0.56,0.19,2.0,0.081,17.0,108.0,0.99620,3.32,0.54,9.5
3411,6.0,0.14,0.17,5.6,0.036,37.0,127.0,0.99373,3.05,0.57,9.8
2370,6.3,0.10,0.24,6.0,0.039,25.0,107.0,0.99511,3.59,0.49,10.5
454,6.4,0.28,0.29,1.6,0.052,34.0,127.0,0.99290,3.48,0.56,10.5
4590,6.4,0.33,0.30,7.2,0.041,42.0,168.0,0.99331,3.22,0.49,11.1


# Pre-processing

Scale the data using LabelEncoder and MinMaxScaler

In [9]:
# Scale your data - see Activity 03/05
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [12]:
y_train_categorical.shape

(4872, 7)

# Train the Model

In [17]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=11))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=7, activation='softmax'))

In [18]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               1200      
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 707       
Total params: 12,007
Trainable params: 12,007
Non-trainable params: 0
_________________________________________________________________


In [20]:
# set early stopping as callback
callbacks = [EarlyStopping(monitor='val_loss', patience=2)]
model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=callbacks,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
 - 2s - loss: 1.3130 - accuracy: 0.4717
Epoch 2/60


C:\Users\kenne\Anaconda3\envs\PythonData\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 0s - loss: 1.1374 - accuracy: 0.5201
Epoch 3/60
 - 0s - loss: 1.1086 - accuracy: 0.5394
Epoch 4/60
 - 0s - loss: 1.0969 - accuracy: 0.5435
Epoch 5/60
 - 0s - loss: 1.0887 - accuracy: 0.5339
Epoch 6/60
 - 0s - loss: 1.0824 - accuracy: 0.5392
Epoch 7/60
 - 0s - loss: 1.0768 - accuracy: 0.5470
Epoch 8/60
 - 0s - loss: 1.0745 - accuracy: 0.5511
Epoch 9/60
 - 0s - loss: 1.0690 - accuracy: 0.5550
Epoch 10/60
 - 0s - loss: 1.0678 - accuracy: 0.5454
Epoch 11/60
 - 0s - loss: 1.0604 - accuracy: 0.5546
Epoch 12/60
 - 0s - loss: 1.0573 - accuracy: 0.5536
Epoch 13/60
 - 0s - loss: 1.0544 - accuracy: 0.5589
Epoch 14/60
 - 0s - loss: 1.0547 - accuracy: 0.5513
Epoch 15/60
 - 0s - loss: 1.0473 - accuracy: 0.5599
Epoch 16/60
 - 0s - loss: 1.0451 - accuracy: 0.5589
Epoch 17/60
 - 0s - loss: 1.0399 - accuracy: 0.5616
Epoch 18/60
 - 0s - loss: 1.0371 - accuracy: 0.5591
Epoch 19/60
 - 0s - loss: 1.0348 - accuracy: 0.5597
Epoch 20/60
 - 0s - loss: 1.0337 - accuracy: 0.5642
Epoch 21/60
 - 0s - loss: 1.030

In [21]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 1.0246131682029138, Accuracy: 0.564307689666748


In [22]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [23]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [5 5 6 6 6]
Actual Labels: [4, 5, 6, 6, 8]


# Save the Model

In [24]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'deeplearning.sav'
joblib.dump(model, filename)

['deeplearning.sav']